# [Lv4-Day1-Lab1] API 직접 다루기: The Mechanics of Agent Tools

### 실습 목표
LLM Agent의 강력한 기능인 'Tool 사용'의 내부 동작 원리를 완벽하게 이해하기 위해, 이번 시간에는 LLM 없이 순수 Python 코드만으로 외부 시스템과 직접 소통하는 경험을 합니다. 우리는 Agent의 Tool이 내부적으로 수행하는 모든 기계적인(Mechanical) 단계를 직접 구현해 볼 것입니다.

1.  **REST API 직접 호출:** Python의 표준 HTTP 라이브러리인 `requests`를 사용하여, 이론 시간에 배운 REST API(OpenWeatherMap, Finnhub)를 직접 호출합니다.
2.  **JSON 응답 파싱:** API로부터 받은 원시(Raw) JSON 데이터를 Python 딕셔너리로 변환하고, 필요한 정보를 정확히 추출하는 방법을 익힙니다.
3.  **견고한 Tool 함수 설계:** 에러 핸들링을 포함한 실무 수준의 재사용 가능한 함수(Tool의 원형)를 설계합니다.
4.  **Function Calling의 필요성 체감:** 이 모든 수동적인 과정을 직접 경험함으로써, 다음 실습에서 배울 Function Calling이 이 번거로운 작업을 어떻게 자동화하여 우리를 편하게 해주는지 그 가치를 명확히 이해하게 됩니다.

### 0. 사전 준비: 라이브러리 설치 및 API 키 설정
이번 실습에서는 외부 API와 통신하기 위한 `requests`, `finnhub-python` 라이브러리를 설치하고, 1일차 이론 시간에 안내된 API 키들을 설정합니다.

- `finnhub-python`: 실시간 주식 데이터를 제공하는 Finnhub API를 쉽게 사용하기 위한 Python 클라이언트입니다.


In [4]:
# !pip install requests finnhub-python -q

### 0.2. API 키 발급 및 보안 설정
이번 실습에서는 총 3개의 무료 API 키가 필요합니다.


**1. OpenWeatherMap API Key (날씨 정보 Tool용):**
- [OpenWeatherMap](https://openweathermap.org/api)에 가입하고 로그인합니다.
- API Keys 탭에서 Default 키를 복사합니다.
- `OPENWEATHER_API_KEY` 라는 이름으로 저장합니다

**2. Finnhub API Key (주식 정보 Tool용):**
- [Finnhub](https://finnhub.io/register)에 가입하고 로그인합니다.
- 대시보드에서 API Key를 복사합니다. (무료 플랜으로 충분합니다.)
- `FINNHUB_API_KEY` 라는 이름으로 저장합니다

In [5]:
import os
from getpass import getpass

# API 키 설정
if "OPENWEATHER_API_KEY" not in os.environ:
    os.environ["OPENWEATHER_API_KEY"] = getpass("OPENWEATHER_API_KEY를 입력하세요: ")

if "FINNHUB_API_KEY" not in os.environ:
    os.environ["FINNHUB_API_KEY"] = getpass("FINNHUB_API_KEY 키를 입력하세요: ")

print("✅ API 키 설정이 완료되었습니다.")

FINNHUB_API_KEY 키를 입력하세요: ········
✅ API 키 설정이 완료되었습니다.


### 1. OpenWeatherMap API: 단계별 분해
첫 번째로, 날씨 API를 호출하는 과정을 단계별로 분해하여 REST API의 각 구성요소가 코드에서 어떻게 구현되는지 확인합니다.

#### 1.1. 요청(Request) 만들기 및 보내기
이론 시간에 배운 Endpoint, Method(`GET`), Parameters를 조합하여 API 서버로 HTTP 요청을 보냅니다. `requests.get()` 함수가 이 역할을 수행합니다. 실무에서는 요청이 항상 성공하리란 보장이 없으므로, `try-except`와 `response.raise_for_status()`를 사용하여 견고한 에러 핸들링을 구현하는 것이 매우 중요합니다.

In [6]:
import requests
import json

# 1. API 요청에 필요한 구성요소 정의
api_key = os.environ["OPENWEATHER_API_KEY"]
base_url = "http://api.openweathermap.org/data/2.5/weather"
city = "Seoul"

# 2. 파라미터 딕셔너리 생성
params = {
    "q": city,
    "appid": api_key,
    "units": "metric",  # 섭씨 온도로 받기 위한 파라미터
    "lang": "kr",  # 한국어로 설명 받기 위한 파라미터
}

try:
    # 3. requests 라이브러리를 사용하여 GET 요청 보내기
    print(f"-> {base_url} 로 요청을 보냅니다...")
    response = requests.get(base_url, params=params)

    # 4. 응답 상태 코드 확인 (200이 아니면 에러 발생)
    response.raise_for_status()

    print(f"<- 응답 성공! (상태 코드: {response.status_code})")

    # 5. 응답의 원시 텍스트(Raw Text) 확인
    print("\n--- 원시 응답 (Raw Text) ---")
    print(response.text)

except requests.exceptions.HTTPError as http_err:
    print(f"❌ HTTP 오류 발생: {http_err} - API 키가 유효한지 확인하세요.")
except Exception as e:
    print(f"❌ 알 수 없는 오류 발생: {e}")

-> http://api.openweathermap.org/data/2.5/weather 로 요청을 보냅니다...
<- 응답 성공! (상태 코드: 200)

--- 원시 응답 (Raw Text) ---
{"coord":{"lon":126.9778,"lat":37.5683},"weather":[{"id":804,"main":"Clouds","description":"온흐림","icon":"04d"}],"base":"stations","main":{"temp":29.59,"feels_like":31.72,"temp_min":29.59,"temp_max":29.59,"pressure":1013,"humidity":58,"sea_level":1013,"grnd_level":1003},"visibility":10000,"wind":{"speed":2.5,"deg":197,"gust":3.73},"clouds":{"all":100},"dt":1756345570,"sys":{"country":"KR","sunrise":1756328333,"sunset":1756375703},"timezone":32400,"id":1835848,"name":"Seoul","cod":200}


#### 1.2. 응답(Response) 파싱 및 정보 추출
API 서버가 반환한 JSON 형식의 텍스트는 그 자체로는 다루기 어렵습니다. `response.json()` 메소드를 사용하여 이 텍스트를 Python의 딕셔너리(Dictionary) 객체로 변환합니다. 변환된 후에는, 우리가 원하는 정보(온도, 날씨 설명 등)를 딕셔너리의 키를 통해 손쉽게 추출할 수 있습니다.

In [7]:
# 이전 단계에서 성공적으로 `response`를 받았다고 가정합니다.
if "response" in locals() and response.status_code == 200:
    # 1. JSON 텍스트를 Python 딕셔너리로 파싱
    weather_data = response.json()
    print("--- 파싱된 데이터 (Python Dictionary) ---")
    # json.dumps를 사용하여 보기 좋게 출력
    print(json.dumps(weather_data, indent=2, ensure_ascii=False))

    # 2. 필요한 정보 추출
    location = weather_data["name"]
    temperature = weather_data["main"]["temp"]
    description = weather_data["weather"][0]["description"]

    # 3. 추출된 정보를 사용자 친화적인 문장으로 조합
    print("\n--- 최종 추출 정보 ---")
    final_message = f"{location}의 현재 날씨: 기온은 {temperature}°C이며, 하늘은 '{description}' 상태입니다."
    print(final_message)
else:
    print("이전 단계에서 API 요청이 실패하여 이 셀을 실행할 수 없습니다.")

--- 파싱된 데이터 (Python Dictionary) ---
{
  "coord": {
    "lon": 126.9778,
    "lat": 37.5683
  },
  "weather": [
    {
      "id": 804,
      "main": "Clouds",
      "description": "온흐림",
      "icon": "04d"
    }
  ],
  "base": "stations",
  "main": {
    "temp": 29.59,
    "feels_like": 31.72,
    "temp_min": 29.59,
    "temp_max": 29.59,
    "pressure": 1013,
    "humidity": 58,
    "sea_level": 1013,
    "grnd_level": 1003
  },
  "visibility": 10000,
  "wind": {
    "speed": 2.5,
    "deg": 197,
    "gust": 3.73
  },
  "clouds": {
    "all": 100
  },
  "dt": 1756345570,
  "sys": {
    "country": "KR",
    "sunrise": 1756328333,
    "sunset": 1756375703
  },
  "timezone": 32400,
  "id": 1835848,
  "name": "Seoul",
  "cod": 200
}

--- 최종 추출 정보 ---
Seoul의 현재 날씨: 기온은 29.59°C이며, 하늘은 '온흐림' 상태입니다.


#### 1.3. 재사용 가능한 Tool 함수로 만들기
지금까지의 과정을 하나의 함수로 캡슐화하여, 어떤 도시든 이름만 넣으면 날씨 정보를 반환하는 재사용 가능한 함수, 즉 **'Tool의 원형'**을 만듭니다. 이것이 바로 Agent에게 제공될 Tool의 기본 형태입니다.

In [8]:
def get_current_weather(location: str) -> dict:
    """OpenWeatherMap API를 호출하여 특정 도시의 날씨 정보를 반환하는 Tool 함수"""
    api_key = os.environ["OPENWEATHER_API_KEY"]
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {"q": location, "appid": api_key, "units": "metric", "lang": "kr"}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        return {
            "status": "success",
            "location": data["name"],
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"],
        }
    except requests.exceptions.HTTPError as http_err:
        return {"status": "error", "message": f"HTTP 오류: {http_err}"}
    except Exception as e:
        return {"status": "error", "message": f"알 수 없는 오류: {e}"}


# 함수 테스트
print("--- Tool 함수 테스트 ---")
busan_weather = get_current_weather("Busan")
print(busan_weather)

london_weather = get_current_weather("London")
print(london_weather)

--- Tool 함수 테스트 ---
{'status': 'success', 'location': 'Busan', 'temperature': 28.7, 'description': '온흐림'}
{'status': 'success', 'location': 'London', 'temperature': 14.77, 'description': '구름조금'}


### 2. Finnhub API: 반복 숙달
이제 다른 API(Finnhub)에 대해서도 동일한 과정을 반복하며 API 호출 패턴을 완전히 익힙니다. 이번에는 공식 클라이언트 라이브러리(`finnhub-python`)를 사용해 보겠습니다. 이 라이브러리 역시 내부적으로는 `requests`를 사용하여 API를 호출하지만, 개발자가 더 편리하게 사용할 수 있도록 추상화되어 있습니다.

In [9]:
import finnhub


def get_stock_price(symbol: str) -> dict:
    """Finnhub API를 호출하여 특정 주식의 현재가를 반환하는 Tool 함수"""
    try:
        finnhub_client = finnhub.Client(api_key=os.environ["FINNHUB_API_KEY"])
        quote = finnhub_client.quote(symbol)

        # API가 유효하지 않은 심볼에 대해 오류 대신 0을 반환하는 경우가 있음
        if quote.get("c") is None or quote.get("c") == 0:
            return {"status": "error", "message": f"'{symbol}'에 대한 주가 정보를 찾을 수 없습니다."}

        return {
            "status": "success",
            "symbol": symbol,
            "current_price": quote["c"],
            "change": quote["d"],
            "percent_change": quote["dp"],
        }
    except Exception as e:
        return {"status": "error", "message": f"API 호출 중 오류 발생: {e}"}


# 함수 테스트
print("--- Tool 함수 테스트 ---")
apple_stock = get_stock_price("AAPL")
print(f"Apple 주가: {apple_stock}")

samsung_stock = get_stock_price("TQQQ")  # Finnhub는 한국 주식 심볼도 지원 (Ticker.Market)
print(f"삼성전자 주가: {samsung_stock}")

invalid_stock = get_stock_price("INVALID_SYMBOL")
print(f"유효하지 않은 심볼: {invalid_stock}")

--- Tool 함수 테스트 ---
Apple 주가: {'status': 'success', 'symbol': 'AAPL', 'current_price': 230.49, 'change': 1.18, 'percent_change': 0.5146}
삼성전자 주가: {'status': 'error', 'message': "API 호출 중 오류 발생: FinnhubAPIException(status_code: 403): You don't have access to this resource."}
유효하지 않은 심볼: {'status': 'error', 'message': "'INVALID_SYMBOL'에 대한 주가 정보를 찾을 수 없습니다."}


### 3. 결론 및 다음 단계로의 전환

**축하합니다!** 여러분은 이제 LLM 없이도 외부 시스템과 소통하는 Agent의 '팔과 다리'를 직접 만들 수 있게 되었습니다.

**이번 실습을 통해 우리가 겪은 '불편함'을 되짚어 봅시다:**
1.  API 명세서를 직접 읽고 **어떤 Endpoint와 Parameter를 사용해야 할지** 우리가 직접 알아내야 했습니다.
2.  API 응답의 복잡한 **JSON 구조를 미리 파악하고**, 어떤 키에 원하는 정보가 들어있는지 직접 코드로 작성해야 했습니다.
3.  사용자의 다양한 자연어 질문("서울 날씨", "seoul weather", "서울 기온은?")을 해석하여 `get_current_weather("Seoul")` 이라는 **정확한 함수 호출로 변환하는 로직**이 없었습니다.

**다음 단계 예고:**
바로 이 1, 2, 3번의 지루하고 기계적인 작업을 **LLM의 강력한 자연어 이해 능력으로 자동화**하는 것이 바로 **Function Calling** 입니다. Lab 2에서는 오늘 만든 이 Tool 함수들을 그대로 사용하여, LLM이 어떻게 스스로 이 함수들을 호출하고 그 결과를 해석하는지 학습하게 될 것입니다.